In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(" ") #troque o caminho do ficheiro 
df_original = df.copy()

print("\n VALORES NULOS")
print(df.isnull().sum())

print("ESTATÍSTICAS GERAIS")
print(df.describe().T)

plt.figure(figsize=(8, 5))
sns.heatmap(df[['CaixasMultibanco', 'DepositosBancarios']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlação entre variáveis')
plt.show()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.boxplot(x=df['CaixasMultibanco'])
plt.title("Boxplot - Caixas Multibanco")

plt.subplot(1, 2, 2)
sns.boxplot(x=df['DepositosBancarios'])
plt.title("Boxplot - Depósitos Bancários")
plt.show()

# Limpeza e Pré-processamento
# Remover linhas com valores nulos
df = df.dropna()

# Remover outliers com base no IQR
def remover_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    filtro = (df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)
    return filtro

filtro_caixas = remover_outliers('CaixasMultibanco')
filtro_depositos = remover_outliers('DepositosBancarios')
df = df[filtro_caixas & filtro_depositos]

# Análise Não Supervisionada – K-Means
# Agrupar municípios por média das variáveis
agrupado = df.groupby('Município')[['CaixasMultibanco', 'DepositosBancarios']].mean()

# Normalizar os dados
scaler = StandardScaler()
dados_normalizados = scaler.fit_transform(agrupado)

# Aplicar K-Means
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
agrupado['Cluster'] = kmeans.fit_predict(dados_normalizados)

# Visualização dos clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=agrupado['CaixasMultibanco'],
    y=agrupado['DepositosBancarios'],
    hue=agrupado['Cluster'],
    palette='Set2'
)
plt.title("Clusters de Municípios por Estatísticas Bancárias")
plt.xlabel("Caixas Multibanco (média)")
plt.ylabel("Depósitos Bancários (média €)")
plt.legend(title="Cluster")
plt.grid(True)
plt.show()

# Estatísticas Traduzidas
TRADUCAO = {
    'count': 'Quantidade',
    'mean': 'Média',
    'std': 'Desvio Padrão',
    'min': 'Mínimo',
    '25%': '1º Quartil',
    '50%': 'Mediana',
    '75%': '3º Quartil',
    'max': 'Máximo'
}

def traduzir_estatisticas(serie):
    estatisticas = serie.describe()
    estatisticas.index = estatisticas.index.map(lambda x: TRADUCAO.get(x, x))
    return estatisticas


# Análise Individual por Município
def analisar_municipio(nome_municipio):
    nome_municipio = nome_municipio.strip().lower()
    municipios_disponiveis = df_original['Município'].dropna().unique()
    municipios_corretos = [m for m in municipios_disponiveis if m.lower() == nome_municipio]

    if not municipios_corretos:
        print(f"\nMunicípio '{nome_municipio}' não encontrado.")
        return

    nome_exato = municipios_corretos[0]
    municipio_df = df_original[df_original['Município'].str.lower() == nome_exato.lower()]


    print(f"ANÁLISE PARA: {nome_exato}")
    print(f"Período: {municipio_df['Ano'].min()} a {municipio_df['Ano'].max()}")
    print(f"Registros encontrados: {len(municipio_df)}")

    print("\nCAIXAS MULTIBANCO:")
    print(traduzir_estatisticas(municipio_df['CaixasMultibanco']).to_string())

    print("\nDEPÓSITOS BANCÁRIOS:")
    print(traduzir_estatisticas(municipio_df['DepositosBancarios']).to_string())

    municipio_df = municipio_df.sort_values('Ano')

    # Gráficos de evolução
    plt.figure(figsize=(14, 6))
    plt.suptitle(f"Análise para {nome_exato}", y=1.05)

    plt.subplot(1, 2, 1)
    plt.plot(municipio_df['Ano'], municipio_df['CaixasMultibanco'], 'bo-')
    plt.title('Evolução das Caixas Multibanco')
    plt.xlabel('Ano')
    plt.ylabel('Quantidade')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(municipio_df['Ano'], municipio_df['DepositosBancarios'], 'ro-')
    plt.title('Evolução dos Depósitos Bancários')
    plt.xlabel('Ano')
    plt.ylabel('Valor (€)')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    media_nacional = df.groupby('Ano').agg({
        'CaixasMultibanco': 'mean',
        'DepositosBancarios': 'mean'
    }).rename(columns={
        'CaixasMultibanco': 'Caixas_Média_Nacional',
        'DepositosBancarios': 'Depósitos_Média_Nacional'
    })

    dados_municipio = municipio_df.groupby('Ano').agg({
        'CaixasMultibanco': 'mean',
        'DepositosBancarios': 'mean'
    }).rename(columns={
        'CaixasMultibanco': 'Caixas_Município',
        'DepositosBancarios': 'Depósitos_Município'
    })

    comparacao = dados_municipio.join(media_nacional)
    print("\nCOMPARAÇÃO COM A MÉDIA NACIONAL:")
    print(comparacao.to_string(float_format="%.2f"))

# Interface de Utilizador
while True:
    print("\nSISTEMA DE ANÁLISE DE ESTATÍSTICAS ECONÔMICAS POR MUNICÍPIO")
    print("Digite o nome de um município para analisar (ou 'sair' para encerrar)")

    nome = input("Nome do município: ").strip()

    if nome.lower() == 'sair':
        break

    analisar_municipio(nome)


FileNotFoundError: [Errno 2] No such file or directory: '/dados_municipais_integrados_limpos.csv'